In [1]:
import numpy as np
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
from jax import config
config.update("jax_enable_x64", True)

# Is there a sweet spot for the final time $\tau$ so that the error is not so small that is close to machine precision but not that big either?

In [2]:
from opentn.transformations import create_kitaev_liouvillians, exp_operator_dt, super2ortho, create_trotter_layers
d, N, gamma = 2, 4, 1
tau = 4
Lvec, Lvec_odd, Lvec_even, Lnn = create_kitaev_liouvillians(N=N, d=d, gamma=gamma, pbc=True)
exp_Lvec, exp_Lvec_odd, exp_Lvec_even = create_trotter_layers([Lvec, Lvec_odd, Lvec_even], tau=tau)

In [3]:
from opentn.optimization import frobenius_norm, model_stiefel_local, get_kitaev_trotter_local_ansatz
from opentn.stiefel import gradient_stiefel_vec, riemannian_hessian_vec

xs_timestep_1_tau_4 = [super2ortho(x.real, rank=2) for x in get_kitaev_trotter_local_ansatz(gamma, tau, n=1)]

f_stiefel = lambda xi: frobenius_norm(model_stiefel_local(xi, N, d, ), exp_Lvec)
grad_stiefel = lambda xi: gradient_stiefel_vec(xi, f_stiefel, metric='canonical')
hessian_stiefel = lambda xi: riemannian_hessian_vec(xi, f_stiefel, metric='canonical')

In [4]:
# initial cost function with stiefel ansatz for two sites
f_stiefel(xs_timestep_1_tau_4)

DeviceArray(0.14312676, dtype=float64)

In [ ]:
# trust rergion optimization functions
from opentn.trust_region_rcopt import riemannian_trust_region_optimize
from opentn.stiefel import retract_stiefel